<a href="https://colab.research.google.com/github/vishalbalaji-v/Personal-Projects/blob/main/Guided%20Projects%20/%20Rossman%20Kaggle%3A%20Forecasting%20Sales%20/%20Part%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML-2: Trees, Model Interrogation and Bayesian Workflow
# Homework 2: Rossman Kaggle: Forecasting Sales
# Part 4 : Modelling with embeddings!
**ML-2 Cohort 1** <br>
**Instructor: Dr. Rahul Dave**<br>
**Max Score: 100** <br>

In [ ]:
#importing libraries
import numpy as np
import scipy.stats
import scipy.special
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
import pandas as pd
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers.embeddings import Embedding
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import pickle
import csv
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

We will repeat the first initial steps again from Part 2

Lets import the feature_train_data.pickle file and set X,y values from the pickle file

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
f = open('/content/drive/MyDrive/Colab files/Homework 2 - ML2/feature_train_data.pickle','rb')
(X, y) = pickle.load(f)

In [ ]:
# we will split the train_ratio and 90% and 10% and set the train_size
train_ratio = 0.9
num_records = len(X)
train_size = int(train_ratio * num_records)

In [ ]:
#lets look at our data
X[1], y[1]

(array([   0,    0,    0, 1058,    1,    0,    0,    1]), 4491)

The next set of inputs is following: Write the code for this yourself

1. Do you want to one hot encode the data?
2. Do you want to provide embeddings as input - this will be set to True for models with entity embeddings
3. Do you want to save the emmbeddings? - again set to true if you want to entity embeddings
4. if 3 is set to true, we want to save them to a embeddings.pickle


In [ ]:
#your code here
one_hot_as_input = False 
embeddings_as_input = True #embeddings later on needs to set to true for Part 3
save_embeddings = True

saved_embeddings_fname = "/content/drive/MyDrive/Colab files/Homework 2 - ML2/embeddings.pickle"  # set save_embeddings to True to create this file

## Now lets work with Models with Entity embedding!!

Now that you have saved embeddings - push this into the other models as an input with X. 

How will we do this? 

We need to update our X values. 

1. We will define a function called embed_features, which will combine the embedding with X. 
2. Call this function and update it with the inital X values taken from the pickle file - features_train_data
3. Then split you data, X_emb - into Xtrain and X_Val, y_train and y_val remain the same
4. Sample the data

In [ ]:
#combining embedding
def embed_features(X, saved_embeddings_fname):
    f_embeddings = open(saved_embeddings_fname, "rb")
    embeddings = pickle.load(f_embeddings) # loading embedding

    index_embedding_mapping = {3: 0, 4: 1, 0: 2, 1: 3, 2: 4, 7: 5}
    X_embedded = []

    (num_records, num_features) = X.shape # rows and columns
    for record in X: # each row or record
        embedded_features = [] 
        for i, feat in enumerate(record): # index and the value of each index of the list - we have to think about the ordering
            feat = int(feat)
            if i not in index_embedding_mapping.keys():
                embedded_features += [feat]
            else:
                embedding_index = index_embedding_mapping[i]
                embedded_features += embeddings[embedding_index][feat].tolist()

        X_embedded.append(embedded_features)

    return np.array(X_embedded)

**Explain what is happening the function above**

your answer here

### Embedding with X - input

In [ ]:
#check if embedding is needed, if yes call embed_features - with X and the embeddings passed to it - call this new X as X_emb
if embeddings_as_input:
  #your code here 
  X_emb = embed_features(X, saved_embeddings_fname)

Split the train and validation based on train size and on the new X_emb values from the previous code

In [ ]:
#update the X_train and X_val
#your code here
X_train, X_val, y_train, y_val = train_test_split(X_emb,y, train_size=train_ratio, random_state = 42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(759904, 42) (84434, 42) (759904,) (84434,)


In [ ]:
def sample(X, y, n):
    '''random samples'''
    num_row = X.shape[0]
    indices = np.random.randint(num_row, size=n)
    return X[indices, :], y[indices]

In [ ]:
X_train, y_train = sample(X_train, y_train, 200000)  # Simulate data sparsity
print("Number of samples used for training: " + str(y_train.shape[0]))

Number of samples used for training: 200000


## Add the embeddings into the Models and check their MAPE!

All the models defined here will have the same parameters as the ones defined in Part 2!

In [ ]:
#defining mape
def MAPE(Y_actual,Y_Predicted):
    #your code here
    error = []
    Y_actual = np.log(Y_actual)
    Y_Predicted = np.log(Y_Predicted)
    for i in range(len(Y_actual)):
      error.append(abs((Y_actual[i] - Y_Predicted[i])/Y_actual[i]))
    return(np.mean(error))

### Model 1: Random Forests

In [ ]:
#your code here
forest = RandomForestRegressor(n_estimators=200,max_depth=35,min_samples_split=2,min_samples_leaf=1)
forest.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=35, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
forest_MAPE_train = MAPE(y_train,forest.predict(X_train))
forest_MAPE_test = MAPE(y_val,forest_pred)

In [ ]:
print('Training Mean Absolute Percentage Error is {:.4f}'.format(forest_MAPE_train))
print('Testing Mean Absolute Percentage Error is {:.4f}'.format(forest_MAPE_test))

Training Mean Absolute Percentage Error is 0.0032
Testing Mean Absolute Percentage Error is 0.0100


### Model 2: Boosting

In [ ]:
#your code here
#your code here
feature_Xtr = xgb.DMatrix(X_train, label=y_train)
feature_Xval = xgb.DMatrix(X_val)

In [ ]:
param_grid = {
    'nthread': -1,
    'max_depth': 7,
    'eta': 0.02,
    'silent': 1,
    'objective': 'reg:linear',
    'colsample_bytree': 0.7,
    'subsample': 0.7
}
num_round = 3000

In [ ]:
xgb_fit = xgb.train(param_grid,feature_Xtr,num_round)

In [ ]:
xgb_pred = xgb_fit.predict(feature_Xval)

In [ ]:
xgb_MAPE_train = MAPE(y_train,xgb_fit.predict(feature_Xtr))
xgb_MAPE_test = MAPE(y_val,xgb_pred)

print('Training Mean Absolute Percentage Error is {:.4f}'.format(xgb_MAPE_train))
print('Testing Mean Absolute Percentage Error is {:.4f}'.format(xgb_MAPE_test))

Training Mean Absolute Percentage Error is 0.0068
Testing Mean Absolute Percentage Error is 0.0081


In [ ]:
lst_no_embed = [0.0264, 0.02, 0.0054]
lst_with_embed = [round(forest_MAPE_test,4), round(xgb_MAPE_test,4), '']
df = pd.DataFrame({'MAPE': lst_no_embed, 'MAPE(with EE)': lst_with_embed}, index=['Random Forest', 'XG Boost', 'MLP'])

## Final Commments!

Apart from how long this homework was, lets make some other final comments

**Question 1: Did you models with Entity Embeddings perfom better?**

Yes

**Question 2: Now that you have completed this homework, lets answer the main purpose of the homework - How do you think entity embeddings improved the MAPE score. To show this do a similar table like the one did in Paper**

Entity embeddings force the network to learn the intrinsic properties of each of the feature as well as the sales distribution in the feature space. It encodes the data in a more continuous form by mapping closely related values in an embedding space, thereby giving better predictions


**Question 3: Add a table here to show the similar to the one done in paper - to show the different MAPE values with and without embeddings**



In [ ]:
df

,MAPE,MAPE(with EE)
Random Forest,0.0264,0.01
XG Boost,0.0200,0.0081
MLP,0.0054,
